# Action Detection

## Set up enviornment

In [1]:
!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
    100% |████████████████████████████████| 484.0MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b12a000 @  0x7f56336941c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 9.7MB/s 


    100% |████████████████████████████████| 2.0MB 14.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision
import numpy as np
import scipy
from scipy.io import loadmat
from random import shuffle
from torch.utils.data import Dataset, DataLoader
import cv2
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [0]:
os.chdir('drive/Colab_Notebooks/c3d/')

## Data Loader

In [4]:
video_rootdir="./ReCompress_Videos"
mask_rootdir="./puppet_mask"
pose_rootdir="./joint_positions"

video_pathes=[]
mask_pathes=[]
pose_pathes=[]
for root, dirs, files in os.walk(video_rootdir):
    for file in files:
        video_pathes.append(os.path.join(root, file))

for root, dirs, files in os.walk(mask_rootdir):
    for file in files:
        mask_pathes.append(os.path.join(root, file)) 

for root, dirs, files in os.walk(pose_rootdir):
    for file in files:
        pose_pathes.append(os.path.join(root, file))
        
video_pathes = sorted(video_pathes)
mask_pathes = sorted(mask_pathes)
pose_pathes = sorted(pose_pathes)

KeyboardInterrupt: ignored

In [0]:
video_pathes_train, video_pathes_valid, mask_pathes_train, mask_pathes_valid, pose_pathes_train, pose_pathes_valid = train_test_split(video_pathes, mask_pathes, pose_pathes, test_size=0.3)
class_names=[name for name in os.listdir(video_rootdir)]
class_num=len(class_names)

In [0]:
class JHMDB(torch.utils.data.Dataset):
    def __init__(self, video_pathes, mask_pathes, pose_pathes, class_names, F = 15):
        
        self.data = {'video': [], 'label': [], 'mask':[], 'pose':[], 'scale':[]}
        
        self.classdict = {}
        for i, x in enumerate(class_names):
            self.classdict[x] = i

        video_num=len(video_pathes)
        mask_num=len(mask_pathes)

        for i in range(video_num):
            video=[]
            cap = cv2.VideoCapture(video_pathes[i])
            has_frame=True
            while(has_frame):
                _, frame = cap.read()
                has_frame = frame is not None

                if has_frame:
                    frame = cv2.resize(frame, (112, 112), interpolation = cv2.INTER_CUBIC)
                    video.append(frame)
            cap.release()
                    
            self.data['video'].append(video)

            mask_mat = loadmat(mask_pathes[i]) 
            masks = cv2.resize(mask_mat['part_mask'], (112, 112), interpolation = cv2.INTER_CUBIC) # (112, 112, F)
            self.data['mask'].append(masks)
            
            self.data['label'].append(video_pathes[i].split('/')[-2])
            
            pose_mat = loadmat(pose_pathes[i])['pos_img']
            scale = loadmat(pose_pathes[i])['scale']
            self.data['pose'].append(pose_mat)
            self.data['scale'].append(scale[0]) # redundant dim
    
    def _compute_mean(self):
        meanstd_file = './data/jhmdbmean'
        if os.path.isfile(meanstd_file):
            meanstd = torch.load(meanstd_file)
        else:
            mean = torch.zeros(3)
            std = torch.zeros(3)
            for videos in self.data['video']:
                for img in videos:
                    # CxHxW
                    mean += np.reshape(img, (img.shape(0), -1)).mean(1)
                    std += np.reshape(img, (img.shape(0), -1)).std(1)
            mean /= len(self.train)
            std /= len(self.train)
            meanstd = {
                'mean': mean,
                'std': std,
                }
            torch.save(meanstd, meanstd_file)
        if self.is_train:
            print('    Mean: %.4f, %.4f, %.4f' % (meanstd['mean'][0], meanstd['mean'][1], meanstd['mean'][2]))
            print('    Std:  %.4f, %.4f, %.4f' % (meanstd['std'][0], meanstd['std'][1], meanstd['std'][2]))
            
        return meanstd['mean'], meanstd['std']
        
    def __getitem__(self, index):
        # video (C, F, 112, 112)
        # mask (112, 112, F)
        # pose (2, 15, F)
        # scale (F)
        # label (1), 
        # randomly select 15 consecutive frames (F = 15)
        F = 15
        frame_num = self.data['mask'][index].shape[2]
        frame_shuffle=list(range(frame_num))
        shuffle(frame_shuffle)
        selected_frame=sorted(frame_shuffle[0:F])
       
        # change pose position according to resize
        pose_data = torch.from_numpy(self.data['pose'][index][:,:,selected_frame].astype('float'))
        pose_data[0,:,:] = pose_data[0,:,:] * 112 / 240
        pose_data[1,:,:] = pose_data[1,:,:] * 112 / 320
        
        
        video = np.transpose(np.array(self.data['video'][index])[selected_frame,:,:,:], (3, 0, 1, 2))
        return torch.FloatTensor(video), \
            torch.from_numpy(self.data['mask'][index][:,:,selected_frame].astype('float')), \
            pose_data, \
            torch.from_numpy(self.data['scale'][index][selected_frame].astype('float')), \
            torch.LongTensor([self.classdict[self.data['label'][index]]])
        
    def __len__(self):
        return len(self.data['scale'])

In [0]:
train_dataset = JHMDB(video_pathes_train, mask_pathes_train, pose_pathes_train, class_names)
valid_dataset = JHMDB(video_pathes_valid, mask_pathes_valid, pose_pathes_valid, class_names)

In [0]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=10, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=10, shuffle=False)

## Model

N \* 3 \* 15 \* 112 \* 112 --> N \* 64 \* 15 \* 56 \* 56 -->  N \* 128 \* 8 \* 28 \* 28 --> N \* 256 \* 4 \* 14 \* 14 --> N \* 512 \* 2 \* 7 \* 7 --> N \* 512 \* 1 \* 4 \* 4

In [0]:
class C3D(nn.Module):
    """
    The C3D network as described in [1].
    """

    def __init__(self,class_num):
        super(C3D, self).__init__()

        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(1, 0, 0))

        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))

        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, class_num)

        self.dropout = nn.Dropout(p=0.5)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, x):

        h = self.relu(self.conv1(x))
        h = self.pool1(h)

        h = self.relu(self.conv2(h))
        h = self.pool2(h)

        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.pool3(h)

        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.pool4(h)

        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)

        h = h.view(-1, 8192)
        h = self.relu(self.fc6(h))
        h = self.dropout(h)
        h = self.relu(self.fc7(h))
        h = self.dropout(h)

        logits = self.fc8(h)

        return logits

## Train

In [0]:
def eval_video(net,test_video,test_label):
    net.eval()
    _, indices=torch.max(net(test_video),1)
    correct_prediction=torch.eq(indices,test_label.squeeze(1)).data.sum()
    return correct_prediction.cpu().numpy()

In [0]:
criterion = nn.CrossEntropyLoss().cuda()
net=C3D(class_num).cuda()
print(net)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(1, 0, 0), dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3),

In [0]:
def train(epoch,step,disp_interval):
    net.train()
    train_loss = 0
    step_cnt = 0
    for index,x in enumerate(train_loader):
        train_video=Variable(x[0]).cuda()
        train_video_label=Variable(x[4]).cuda()
        train_result=net(train_video)
        
        # forward
        loss = criterion(train_result,train_video_label.squeeze(1))
        train_loss += loss.data[0]
        step_cnt += train_video_label.shape[0]
        step += train_video_label.shape[0]
        # backward pass and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Log to screen
        if step % disp_interval == 0:
            log_text = 'epoch %d, step %d, loss: %.4f' % (epoch, step, train_loss / step_cnt)
            print(log_text)
            train_loss = 0
            step_cnt = 0
        
        # Evaluate the model
        if step % eval_interval == 0:
            test_size=0
            correct_num=0
            for y in train_loader:
                test_video=Variable(y[0]).cuda()
                test_video_label=Variable(y[4]).cuda()
                test_size += test_video_label.shape[0]
                correct_num += eval_video(net,test_video,test_video_label)
            accuracy = correct_num/test_size*100 
            print(accuracy) 
            
    return step

In [0]:
step=0
disp_interval=10
eval_interval=100

for epoch in range(100):
    step += train(epoch,step,disp_interval)